In [1]:
import pandas as pd
import os
import time
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from model import LineageMantaObjectOptimize
from utils import create_raw_table
from config import NODE_PATH, LINEAGE_SOURCE, LINEAGE_TARGET, EXPORT_MANTA_FILE_PATH

In [2]:
if __name__ == '__main__':
    # start_time = time.time()
    object = LineageMantaObjectOptimize(lineage_name= 'manta')
    df_raw, table_name_list = create_raw_table(object)
    df_graph = None
    pd.set_option('display.max_rows', None)
    for table_name in table_name_list:
        # time.sleep(5)
        df = df_raw.query(f"Table_name == '{table_name}'")
        df_describe_op = object.describe_table_optimized(
            df_path=df,
            source_col=LINEAGE_SOURCE,
            target_col=LINEAGE_TARGET,
        )
        all_values = list(df_describe_op.get("all_values"))
        redact_node_df = object.redact_node_with_int(total_nodes_df=df)
        combo_list = redact_node_df[NODE_PATH].tolist()
        # combo_list = [1,2,1868,72,824,928,40,121,366,1691,35,548,663
		# ,1944,509,638,1800,620,750,54,1509,2166,1531,1173,1013,74,345,245
		# ,762,2165,1439,1525,75,631,840,1734,1738,2001,728,1728,1987,1341,145,1724,22,282]
        dictionary_op = object.mapping_dict_nodepath_optimize(df =redact_node_df)
        adjacency_list = object.create_adjacency_list_optimized(dictionary = dictionary_op)
        # print(dictionary_op)
        # print(adjacency_list)
        graph = object.deploy_lineage_manta_optimized(df_raw= df, 
                                    combo_list_copy=combo_list, 
                                    dictionary=dictionary_op,
                                    adjacency_list = adjacency_list, 
                                    source_col=LINEAGE_SOURCE, 
                                    target_col=LINEAGE_TARGET)
        if df_graph is None:
            df_graph = graph
        else:
            df_graph = pd.concat([df_graph, graph])

LINEAGE_MANTA_TEST
TABLE_NAME =  ACCOUNT | FILE_NAME=  relations_ACCOUNT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_ACCOUNT.csv


In [ ]:
# df_graph = graph

In [ ]:
print(len(df_graph))

In [ ]:
# for item in df_graph:
#     print(item)

In [ ]:
# graph_list = list (filter(lambda f: not any(set(f) < set(g) for g in graph), graph) )
# print(len(graph_list))

In [ ]:
# for item in df_graph[100:1000]:
#     print(item)

In [ ]:
print(df_graph)

In [ ]:
df_graph.to_csv(f'{EXPORT_MANTA_FILE_PATH}/df_graph_no_duplicate.csv', index = False)